# I -  Lecture de fichier

In [22]:
import codecs
import re
import numpy as np
import sklearn.feature_extraction.text as txt

def read_file(fn):
    with codecs.open(fn,encoding="utf-8") as f:
        return f.read()

def compteLignes(fname):
    fd = open(fname, 'r')
    n = 0
    for line in fd:
        n += 1
    return n

def file_array(fname, nblignes) : 
    
    #nblignes = compteLignes(fname)
    #print("nblignes = %d"%nblignes)

    alltxts = []
    labs = np.ones(nblignes)
    s = codecs.open(fname, 'r','utf-8') # pour régler le codage

    cpt = 0
    for i in range(nblignes):
        txt = s.readline()

        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)

        if lab.count('M') >0:
            labs[cpt] = -1
        alltxts.append(txt)

        cpt += 1
        
    x = np.array(alltxts)
    y = np.array(labs)
    
    return x[:nblignes], y[:nblignes]

In [45]:
# data train & test

n_train = 10000
n_test = 100

fname_train = "AFDpresidentutf8/corpus.tache1.learn.utf8"
fname_test = "AFDpresidentutf8/corpus.tache1.test.utf8"

x_train, y_train = file_array(fname_train, n_train)
x_test, y_test = file_array(fname_test, n_test)

print(x_train)

[" Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.\n"
 " D'abord merci de cet exceptionnel accueil que les Congolais, les Brazavillois, nous ont réservé cet après-midi.\n"
 " C'est toujours très émouvant de venir en Afrique car c'est probablement l'une des rares terres du monde où l'on ait conservé cette convivialité, cette amitié, ce respect de l'autre qui s'expriment avec chaleur, avec spontanéité et qui réchauffent le coeur de ceux qui arrivent et de ceux qui reçoivent.\n"
 ... " Ils m'ont fait part de leurs expériences.\n"
 " Des expériences qui montrent que l'imagination, l'esprit d'équipe, l'engagement de tous peuvent faire reculer la violence.\n"
 " Ces expériences, il faut les faire mieux connaître afin que les responsables éducatifs puissent s'en inspirer.\n"]


# II - Création de vecteurs

In [46]:
vec = txt.CountVectorizer() # outil de vectorisation

# train
bow_train = vec.fit_transform(x_train) # instatiation de l'outil de vectorisation sur le corpus

# test
bow_test = vec.fit_transform(x_test) # instatiation de l'outil de vectorisation sur le corpus

print(bow_test)

  (0, 764)	1
  (0, 340)	1
  (0, 104)	1
  (0, 523)	1
  (0, 692)	1
  (0, 289)	1
  (0, 339)	1
  (0, 215)	1
  (0, 451)	2
  (0, 453)	1
  (0, 204)	1
  (0, 407)	1
  (0, 539)	1
  (0, 249)	5
  (0, 541)	1
  (0, 180)	2
  (0, 860)	1
  (0, 143)	1
  (0, 567)	1
  (0, 218)	1
  (0, 83)	1
  (0, 429)	1
  (0, 255)	1
  (0, 87)	1
  (0, 382)	1
  :	:
  (98, 249)	1
  (98, 826)	1
  (99, 695)	1
  (99, 270)	1
  (99, 433)	1
  (99, 418)	1
  (99, 84)	1
  (99, 67)	1
  (99, 132)	1
  (99, 4)	1
  (99, 569)	1
  (99, 240)	1
  (99, 18)	1
  (99, 800)	1
  (99, 248)	1
  (99, 749)	1
  (99, 787)	1
  (99, 356)	1
  (99, 767)	2
  (99, 570)	3
  (99, 593)	1
  (99, 617)	1
  (99, 442)	1
  (99, 451)	1
  (99, 453)	1


# III - Classification

## SVM

In [49]:
import numpy as np
import sklearn.naive_bayes as nb
from sklearn import svm
from sklearn import linear_model as lin
from sklearn.model_selection import cross_validate

# SVM
clf = svm.LinearSVC()

scores = cross_validate(clf, bow_train, y_train, cv=5, scoring='f1')

print(scores)

{'fit_time': array([0.19016171, 0.16162992, 0.19480968, 0.106915  , 0.22972012]), 'score_time': array([0.00089407, 0.00082946, 0.00082564, 0.00082922, 0.00096941]), 'test_score': array([0.89875342, 0.89241085, 0.89467312, 0.90003039, 0.88406696]), 'train_score': array([0.99883658, 0.99844889, 0.99875873, 0.99837197, 0.9989917 ])}


## Regresson logistique

In [51]:
clf = lin.LogisticRegression()

scores = cross_validate(clf, bow_train, y_train, cv=5, scoring='f1') # scoring roc_auc

print(scores)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'fit_time': array([0.30883241, 0.09477615, 0.08600235, 0.10614181, 0.07614899]), 'score_time': array([0.00089812, 0.0008564 , 0.0008471 , 0.00084162, 0.00085521]), 'test_score': array([0.91306919, 0.90871492, 0.91058341, 0.91408326, 0.89924357]), 'train_score': array([0.97792326, 0.97739898, 0.97627737, 0.9767265 , 0.97784882])}


## Multinomial

In [52]:
clf = nb.MultinomialNB()

scores = cross_validate(clf, bow_train, y_train, cv=5, scoring='f1')

print(scores)

{'fit_time': array([0.07199526, 0.00467038, 0.0042088 , 0.00369382, 0.00337958]), 'score_time': array([0.00428152, 0.00126743, 0.00111461, 0.00100517, 0.00094724]), 'test_score': array([0.89901698, 0.90295102, 0.90261716, 0.90431061, 0.89053803]), 'train_score': array([0.95727535, 0.95556908, 0.95636972, 0.95542753, 0.95631031])}
